In [1]:
import mysql.connector
import os
import pandas as pd

In [2]:
df = pd.read_csv("/home/antonio/livros_data_set.csv")

In [3]:
df

,Unnamed: 0,Nome,Preco,Categoria,Estrelas,Estoque
0,0,It's Only the Himalayas,45.17,Travel,2,1
1,1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,Travel,4,1
2,2,See America: A Celebration of Our National Par...,48.87,Travel,3,1
3,3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,Travel,2,1
4,4,Under the Tuscan Sun,37.33,Travel,3,1
...,...,...,...,...,...,...
512,512,Why the Right Went Wrong: Conservatism--From G...,52.65,Politics,4,1
513,513,Equal Is Unfair: America's Misguided Fight Aga...,56.86,Politics,1,1
514,514,Amid the Chaos,36.58,Cultural,1,1
515,515,Dark Notes,19.19,Erotica,5,1


In [4]:
df.Nome = df.Nome.str.upper()
df.Categoria = df.Categoria.str.upper()
df.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
df

,Nome,Preco,Categoria,Estrelas,Estoque
0,IT'S ONLY THE HIMALAYAS,45.17,TRAVEL,2,1
1,FULL MOON OVER NOAHÂS ARK: AN ODYSSEY TO MOU...,49.43,TRAVEL,4,1
2,SEE AMERICA: A CELEBRATION OF OUR NATIONAL PAR...,48.87,TRAVEL,3,1
3,VAGABONDING: AN UNCOMMON GUIDE TO THE ART OF L...,36.94,TRAVEL,2,1
4,UNDER THE TUSCAN SUN,37.33,TRAVEL,3,1
...,...,...,...,...,...
512,WHY THE RIGHT WENT WRONG: CONSERVATISM--FROM G...,52.65,POLITICS,4,1
513,EQUAL IS UNFAIR: AMERICA'S MISGUIDED FIGHT AGA...,56.86,POLITICS,1,1
514,AMID THE CHAOS,36.58,CULTURAL,1,1
515,DARK NOTES,19.19,EROTICA,5,1


In [6]:
db_connection = mysql.connector.connect(host='localhost', user='root', password='', database='')
cursor = db_connection.cursor()
sql = "CREATE DATABASE livros_coleta"
cursor.execute(sql)
db_connection.close()

In [7]:
db_connection = mysql.connector.connect(host='localhost', user='root', password='', database='livros_coleta')
cursor = db_connection.cursor()
sql = "CREATE TABLE genero(cod_genero INT PRIMARY KEY AUTO_INCREMENT, nome_genero VARCHAR(95))"
cursor.execute(sql)
db_connection.close()

In [8]:
db_connection = mysql.connector.connect(host='localhost', user='root', password='', database='livros_coleta')
cursor = db_connection.cursor()
sql = "CREATE TABLE livro(cod_livro INT PRIMARY KEY AUTO_INCREMENT,\
                            nome_livro VARCHAR(180),\
                            preco_livro FLOAT,\
                            qtd_estrelas INT,\
                            cod_genero INT,\
                            estoque INT)"
cursor.execute(sql)
db_connection.close()

In [9]:
db_connection = mysql.connector.connect(host='localhost', user='root', password='', database='livros_coleta')
cursor = db_connection.cursor()
sql = "ALTER TABLE livro ADD FOREIGN KEY (cod_genero) REFERENCES genero (cod_genero)"
cursor.execute(sql)
db_connection.close()

In [10]:
db_connection = mysql.connector.connect(host='localhost', user='root', password='', database='livros_coleta')
cursor = db_connection.cursor()
categoria = df['Categoria'].unique().tolist()
for ca in categoria:
    sql = "INSERT INTO genero (nome_genero) VALUES ('{}')".format(ca)
    cursor.execute(sql)
    db_connection.commit()
db_connection.close()

In [14]:
dados = df.values.tolist()
db_connection = mysql.connector.connect(host='localhost', user='root', password='', database='livros_coleta')
cursor = db_connection.cursor()
for Nome,Preco,Categoria,Estrelas,Estoque in dados:
    sql = "SELECT cod_genero FROM genero WHERE nome_genero = '{}'".format(Categoria)
    Nome = Nome.replace('\'','')
    Nome = Nome[0:179]
    cursor.execute(sql)
    cat_cod = cursor.fetchall()
    sql = "INSERT INTO livro (nome_livro,\
            preco_livro,\
            qtd_estrelas,\
            cod_genero, estoque) VALUES ('{}',{},{},{},{})".format(Nome,Preco,Estrelas,cat_cod[0][0],Estoque)
    cursor.execute(sql)
    db_connection.commit()
    
db_connection.close()